# Building ReAct agents with custom tools and Ollama 3 Instruct

## What You'll Learn

In this tutorial, you will learn how to:
1. **Set Up the Environment**: Prepare your development environment with the necessary tools and libraries.
2. **Define Custom Tools**: Create custom functions that can be used by the agent to perform specific tasks.
3. **Integrate with Ollama Model**: Use the Ollama 3 Instruct model to generate responses based on user queries and tool descriptions.
4. **Build a ReAct Agent**: Assemble the defined tools and the language model into a ReAct (Reasoning and Action) agent. This agent will not only process and respond to user queries but will also engage in dynamic reasoning to create, maintain, and adjust action plans. You'll learn how to implement the agent's ability to handle multi-step queries, integrate with external systems, and dynamically adapt its actions based on the context and real-time inputs.

## Basic Concepts

### What are ReAct Agents?

**ReAct (Reasoning and Action)** agents are a type of AI system that combines reasoning capabilities with the ability to take actions. Unlike traditional models that only generate responses based on input data, ReAct agents can use external tools and resources to gather additional information, perform tasks, and make decisions. This dynamic interplay between reasoning and action enables ReAct agents to handle complex, multi-step queries more effectively.

ReAct agents operate by generating reasoning traces (thought processes) and then deciding on actions based on these traces. This allows them to interact with external systems, update their knowledge, and adapt their strategies in real-time.

The basic components of a ReAct agent include:
- 1. A large language model (LLM) that processes input queries and generates reasoning traces.
- 2. A set of tools or APIs that the agent can use to perform specific tasks.
- 3. A framework that integrates the LLM and tools, allowing the agent to reason and act dynamically.

[ReAct: Synergizing Reasoning and Acting in Language Models](https://arxiv.org/abs/2210.03629)

## 1. Setting Up the Environment

Before we dive into building our agent, we need to set up the necessary environment. This involves installing required packages and ensuring our Python environment is ready for development.

In [286]:
# Install termcolor for colored terminal outputs
%pip install termcolor

Note: you may need to restart the kernel to use updated packages.


In [287]:
# Import necessary libraries
from termcolor import colored
import operator
import json
import requests
import os

## 2. Defining the System Prompt and Model Configuration

In this section, we define the system prompt and set up the configuration for the Ollama model. The system prompt provides context and instructions for the model, ensuring it understands how to use the available tools and respond to queries.

### System Prompt Template

The system prompt is a crucial component in guiding the behavior of the language model. It provides a structured way to present the available tools and define how the model should respond to user queries.

*Based on:*
[Model Cards & Prompt formats Llama 3](https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3)

In [288]:
agent_system_prompt_template = """
You are designed to help with a variety of tasks, from answering questions \
    to providing summaries to other types of analyses.

## Tools
You have access to a wide variety of tools. You are responsible for using
the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools
to complete each subtask.

You have access to the following tools:

{tool_descriptions}

## Output Format
To answer the question, please use the following format.

```
Thought: I need to use a tool to help me answer the question.
Action: tool name (one of the available) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 5}}.

If this format is used, the user will respond in the following format:

```
Observation: tool response
```

You should keep repeating the above format until you have enough information
to answer the question without using any more tools. At that point, you MUST respond
in the one of the following two formats:

```
Thought: I can answer without using any more tools.
Answer: [your answer here]
```

```
Thought: I cannot answer the question with the provided tools.
Answer: Sorry, I cannot answer your query.
```

## Additional Rules
- The answer MUST contain a sequence of bullet points that explain how you arrived at the answer. This can include aspects of the previous conversation history.
- You MUST obey the function signature of each tool. Do NOT pass in no arguments if the function expects arguments.
"""

### Setting Up the Ollama Model

The `setup_ollama_model` function configures the model settings, including the endpoint, model name, system prompt, and other parameters. This setup is essential for initializing the model with the correct configuration, ensuring it can process queries and utilize the tools effectively.

In [289]:
def setup_ollama_model(model, system_prompt, temperature=0, stop=None):
    """
    Sets up the Ollama model configuration.

    Parameters:
    model (str): The name of the model to use.
    system_prompt (str): The system prompt to use.
    temperature (float): The temperature setting for the model.
    stop (str): The stop token for the model.

    Returns:
    dict: Configuration for the Ollama model.
    """
    return {
        "model_endpoint": "http://localhost:11434/api/generate",
        "model": model,
        "system_prompt": system_prompt,
        "temperature": temperature,
        "headers": {"Content-Type": "application/json"},
        "stop": stop,
    }


# Example configuration
ollama_config = setup_ollama_model(
    model="llama3:instruct", system_prompt=agent_system_prompt_template
)

## 3. Creating custom tools

This section introduces custom tools that our agent will use to perform specific tasks. These tools are functions designed to handle particular operations, such as mathematical calculations and string manipulation.

### Basic Calculator

The `basic_calculator` function is designed to perform arithmetic operations on two numbers based on the input provided. It accepts a JSON string containing the numbers and the operation type.

In [290]:
def basic_calculator(input_str):
    """
    Perform a numeric operation on two numbers based on the input string.

    Parameters:
    'num1' (int): The first number.
    'num2' (int): The second number.
    'operation' (str): The operation to perform. Supported operations are 'add', 'subtract',
                        'multiply', 'divide', 'floor_divide', 'modulus', 'power', 'lt',
                        'le', 'eq', 'ne', 'ge', 'gt'.

    Returns:
    str: The formatted result of the operation.

    Raises:
    Exception: If an error occurs during the operation (e.g., division by zero).
    ValueError: If an unsupported operation is requested or input is invalid.
    """
    # Clean and parse the input string
    try:
        # Replace single quotes with double quotes
        # input_str_clean = input_str.replace("'", '"')
        # Remove any extraneous characters such as trailing quotes
        # input_str_clean = input_str_clean.strip().strip('"')

        num1 = input_str.get("num1")
        num2 = input_str.get("num2")
        operation = input_str.get("operation")

    except (json.JSONDecodeError, KeyError) as e:
        return str(e), "Invalid input format. Please provide a valid JSON string."

    # Define the supported operations
    operations = {
        "add": operator.add,
        "subtract": operator.sub,
        "multiply": operator.mul,
        "divide": operator.truediv,
        "floor_divide": operator.floordiv,
        "modulus": operator.mod,
        "power": operator.pow,
        "lt": operator.lt,
        "le": operator.le,
        "eq": operator.eq,
        "ne": operator.ne,
        "ge": operator.ge,
        "gt": operator.gt,
    }

    # Check if the operation is supported
    if operation in operations:
        try:
            # Perform the operation
            result = operations[operation](num1, num2)
            result_formatted = (
                f"\n\nThe answer is: {result}.\nCalculated with basic_calculator."
            )
            return result_formatted
        except Exception as e:
            return str(e), "\n\nError during operation execution."
    else:
        return "\n\nUnsupported operation. Please provide a valid operation."

### Reverse String

The `reverse_string` function takes a string as input and returns its reverse. This simple tool is useful for tasks involving string manipulation.

In [291]:
def reverse_string(input_string):
    """
    Reverse the given string.

    Parameters:
    input_string (str): The string to be reversed.

    Returns:
    str: The reversed string.
    """
    # Reverse the string using slicing
    reversed_string = input_string[::-1]

    reversed_string = f"The reversed string is: {reversed_string}\n\nExecuted using the reverse_string function."
    # print (f"DEBUG: reversed_string: {reversed_string}")
    return reversed_string

In [292]:
# To use these tools within our agent, we register them in a list. This list will be referenced by the agent to determine which tools are available for use.
tools = [basic_calculator, reverse_string]

## 4. Building the Agent Class

This section focuses on constructing the `Agent` class, which integrates the tools and the Ollama model. The class handles user queries, determines the appropriate tools to use, and executes the necessary actions. This structure encapsulates the core functionalities of the agent, making it a reusable and maintainable component.

### Agent Class Definition

The `Agent` class is designed to manage the interaction between user queries, the available tools, and the language model. It includes methods for initializing the agent, preparing tool descriptions, generating responses, and executing tools.

- The agent is initialized with a list of tools and a configuration dictionary for the model.
- This includes setting up the model's endpoint, name, system prompt, temperature, headers, and stop token.
- The `prepare_tools` method is called to set up the descriptions of the tools.

**Preparing Tools**

The `prepare_tools` method processes the tools' docstrings to extract and format descriptions and parameter information. This is crucial for generating a system prompt that informs the model of the available tools.

**Generating Model Responses**

The `think` method generates responses by sending user queries and the system prompt to the Ollama model.

**Executing Tools**

The `execute_tool` method determines which tool to use based on the model's response and executes it.

**Main Method: Work**

The `work` method is the primary entry point for processing user queries. It integrates the thinking and execution steps.

In [293]:
class Agent:
    def __init__(self, tools, model_config):
        """
        Initializes the agent with a list of tools and a model configuration.

        Parameters:
        tools (list): List of tool functions.
        model_config (dict): Configuration for the model, including endpoint, model name, system prompt, etc.
        """
        self.tools = tools
        self.model_endpoint = model_config["model_endpoint"]
        self.model_name = model_config["model"]
        self.system_prompt = model_config["system_prompt"]
        self.temperature = model_config["temperature"]
        self.headers = model_config["headers"]
        self.stop = model_config["stop"]
        self.tool_descriptions = self.prepare_tools()
        self.messages: list = []
        if self.system_prompt:
            self.messages.append({"role": "system", "content": self.system_prompt})

    def prepare_tools(self):
        """
        Prepares descriptions of the available tools.

        Returns:
        str: JSON-like formatted string describing the tools.
        """
        tools_info = []

        for tool in self.tools:
            name = tool.__name__
            docstring = (
                tool.__doc__.strip() if tool.__doc__ else "No description available."
            )
            parts = docstring.split("\n\n", 1)
            description = parts[0].split("\n")[0]
            parameters_section = parts[1] if len(parts) > 1 else ""

            param_dict = {}

            if "Parameters:" in parameters_section:
                parameters_lines = parameters_section.split("\n")
                param_section_started = False

                for line in parameters_lines:
                    if "Parameters:" in line:
                        param_section_started = True
                        continue
                    if not param_section_started or not line.strip():
                        continue

                    if ": " in line:
                        param_name_type_desc = line.split(":", 1)
                        if len(param_name_type_desc) == 2:
                            param_name_type, description = param_name_type_desc
                            param_name = param_name_type.split("(", 1)[0].strip()
                            param_type = (
                                param_name_type.split("(", 1)[1].split(")", 1)[0]
                                if "(" in param_name_type
                                else "str"
                            )
                            param_dict[param_name] = {
                                "param_type": param_type.strip(),
                                "description": description.strip(),
                                "required": True,
                            }

            tool_info = {
                "name": name,
                "description": description,
                "parameters": param_dict,
            }
            tools_info.append(tool_info)

        tools_description = "\n".join(
            [
                f"Use the function '{info['name']}' to: {info['description']}\n{json.dumps(info, indent=2)}"
                for info in tools_info
            ]
        )

        return tools_description

    def think(self, prompt):
        """
        Generates a response from the model based on the provided prompt.

        Parameters:
        prompt (str): The user query.

        Returns:
        dict: The response from the model.
        """
        payload = {
            "model": self.model_name,
            "format": "json",
            "prompt": prompt,
            "system": self.system_prompt.format(
                tool_descriptions=self.tool_descriptions
            ),
            "stream": False,
            "temperature": self.temperature,
            "stop": self.stop,
        }

        try:
            response = requests.post(
                self.model_endpoint, headers=self.headers, data=json.dumps(payload)
            )
            response_json = response.json()
            response_dict = json.loads(response_json["response"])

            # print(f"\n\n{response_dict}")
            self.messages.append({"role": "assistant", "content": response_dict})
            return response_dict
        except requests.RequestException as e:
            print(f"Error in invoking model! {str(e)}")
            return {"error": str(e)}

    def work(self, prompt):
        """
        The main method to process a user query, generate a response, and execute the appropriate tool.

        Parameters:
        prompt (str): The user query.

        Returns:
        None
        """
        self.messages.append({"role": "user", "content": prompt})
        result = self.think(prompt)
        return result

In [294]:
# Initialize the agent
agent = Agent(tools=tools, model_config=ollama_config)

## Running the Agent

This section demonstrates how to run the agent with example prompts. We will use the `work` method of the `Agent` class to process different types of user queries, showcasing the agent's ability to select and execute the appropriate tool.

### Calculated

In [295]:
def loop(max_iterations=10, query: str = ""):

    agent = Agent(tools=tools, model_config=ollama_config)

    next_prompt = query

    i = 0

    while i < max_iterations:
        i += 1
        result = agent.work(next_prompt)
        print(result)

        if "Action" in result:
            tool_choice = result["Action"]
            arg = result["Action Input"]

            for tool in tools:
                if tool.__name__ == tool_choice:
                    try:
                        result = tool(arg)
                        next_prompt = f"Observation: {result}"
                        print(colored(result, "cyan"))
                    except Exception as e:
                        print(
                            colored(f"Error executing tool {tool_choice}: {str(e)}", "red")
                        )
                        next_prompt = f"Error executing tool {tool_choice}: {str(e)}"
                    return
            else:
                next_prompt = f"Tool {tool_choice} not found or unsupported operation."
                print(colored(f"Tool {tool_choice} not found or unsupported operation.", "red"))

loop(query="What is 20+20?")
# print(agent.messages)

{'Thought': 'I need to use a tool to help me answer the question.'}
{'Thought': 'I need to use a tool to help me answer the question.'}
{'Thought: I need to use a tool to help me answer the question.': 'Action: basic_calculator'}
{'Thought': 'I need to use a tool to help me answer this question.'}
{'Thought': 'I need to use a basic calculator to find the result.', 'Action': 'basic_calculator', 'Action Input': {'num1': 20, 'num2': 20, 'operation': 'add'}}


The answer is: 40.
Calculated with basic_calculator.
